In [41]:
import LowLevelFEM as FEM
using LowLevelFEM
using SparseArrays

gmsh.initialize()

In [42]:
#gmsh.open("wave_in_rectangle.geo")
gmsh.open("wave_in_rectangle_2.geo")

Info    : Reading 'wave_in_rectangle_2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.000828822s, CPU 0.000828s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.0055138s, CPU 0.005324s)
Info    : 10201 nodes 10407 elements
Info    : Done reading 'wave_in_rectangle_2.geo'


In [43]:
mat = FEM.material("body", E=2.e5, ν=0.3, ρ=7.85e-9)
problem = FEM.Problem([mat], type="PlaneStress")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 201)


LowLevelFEM.Problem("wave_in_rectangle_2", "PlaneStress", 2, [("body", 200000.0, 0.3, 7.85e-9)], 1.0, 10201)

In [44]:
supp1 = FEM.displacementConstraint("left", ux=0)
supp2 = FEM.displacementConstraint("middle", uy=0)
load = FEM.load("source", fx=-1);

In [45]:
dof = problem.non * problem.dim
u0 = zeros(dof)
v0 = zeros(dof);

In [46]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])
C = spzeros(dof, dof)
dof

20402

In [47]:
FEM.applyBoundaryConditions!(problem, K, M, C, f, [supp1, supp2]);

In [48]:
Tₘᵢₙ = FEM.smallestPeriodTime(K, M)

5.92904847515697e-7

In [49]:
u1, v1, t1 = FEM.CDM(K, M, C, f, u0, v0, 2Tₘᵢₙ, Tₘᵢₙ / π)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 1.887274745305393e-7, 3.774549490610786e-7, 5.661824235916178e-7, 7.549098981221572e-7, 9.436373726526965e-7, 1.1323648471832357e-6])

In [50]:
sts = length(t1)
u0 = u1[:, sts]
v0 = v1[:, sts]

f .*= 0
u2, v2, t2 = FEM.CDM(K, M, C, f, u0, v0, 30Tₘᵢₙ, Tₘᵢₙ / π)

([0.0 0.0 … -1.7756843307919433e-6 -4.683301687735873e-6; 0.0 0.0 … -4.9697562664407475e-6 -1.4827030132254326e-6; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -13.554748245954427 -15.406433876024915; 0.0 0.0 … 13.697662955867486 18.47665933054729; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 1.887274745305393e-7, 3.774549490610786e-7, 5.661824235916178e-7, 7.549098981221572e-7, 9.436373726526965e-7, 1.1323648471832357e-6, 1.321092321713775e-6, 1.5098197962443141e-6, 1.6985472707748533e-6  …  1.604183533509585e-5, 1.623056280962639e-5, 1.6419290284156928e-5, 1.6608017758687466e-5, 1.6796745233218004e-5, 1.6985472707748542e-5, 1.717420018227908e-5, 1.736292765680962e-5, 1.7551655131340157e-5, 1.7740382605870695e-5])

In [51]:
gmsh.option.setNumber("General.Trackball", 0)
gmsh.option.setNumber("General.RotationX", 298)
gmsh.option.setNumber("General.RotationY", 346)
gmsh.option.setNumber("General.RotationZ", 332)

In [52]:
vvec1 = FEM.showDoFResults(problem, v1, t=t1, "vvec", name="v1(t)", visible=false)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec1].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec1].VectorType", 5)
gmsh.option.setNumber("View[$vvec1].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec1].NbIso", 20)

In [53]:
vvec2 = FEM.showDoFResults(problem, v2, t=t2, "vvec", name="v2(t)", visible=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec2].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec2].VectorType", 5)
gmsh.option.setNumber("View[$vvec2].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec2].NbIso", 20)

In [54]:
S2 = FEM.solveStress(problem, u2)

LowLevelFEM.StressField([[0.0 0.0 … 0.0 -2.8084380082238363e-9; 0.0 0.0 … 0.0 -3.8667492575266114e-10; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -3.300752643269579e-9; 0.0 0.0 … 0.0 -3.287606563154661e-10; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -6.028353080626717e-9; 0.0 0.0 … 0.0 -6.60925968751474e-10; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -1.1594134384450572e-8; 0.0 0.0 … 0.0 -1.2893407899989747e-9; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -2.2242705963299e-8; 0.0 0.0 … 0.0 -2.4378019021641644e-9; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -4.210441456442558e-8; 0.0 0.0 … 0.0 -4.51380510964074e-9; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -7.845615611538149e-8; 0.0 0.0 … 0.0 -8.209304996764692e-9; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -1.4384902655843103e-7; 0.0 0.0 … 0.0 -1.4678199693296598e-8; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 -2.5952992

In [55]:
s2 = FEM.showStressResults(problem, S2, "seqv", name="σ(t)", t=t2, visible=false, smooth=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$s2].NormalRaise", 100)
#gmsh.option.setNumber("View[$s2].VectorType", 5)
gmsh.option.setNumber("View[$s2].IntervalsType", 3)
gmsh.option.setNumber("View[$s2].NbIso", 20)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [56]:
seqv = FEM.showStressResults(problem, S2, "seqv", name="σeqv(t)", t=t2, visible=false, smooth=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$seqv].NormalRaise", 100)
#gmsh.option.setNumber("View[$seqv].VectorType", 5)
gmsh.option.setNumber("View[$seqv].IntervalsType", 3)
gmsh.option.setNumber("View[$seqv].NbIso", 20)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [57]:
FEM.plotOnPath(problem, "path", step=1im, s2, 500)

4

In [58]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [59]:
dataType, tags, data, time, numComponents = gmsh.view.getModelData(seqv, 0)

("NodeData", UInt64[0x0000000000000001, 0x0000000000000002, 0x0000000000000003, 0x0000000000000004, 0x0000000000000005, 0x0000000000000006, 0x0000000000000007, 0x0000000000000008, 0x0000000000000009, 0x000000000000000a  …  0x00000000000027d0, 0x00000000000027d1, 0x00000000000027d2, 0x00000000000027d3, 0x00000000000027d4, 0x00000000000027d5, 0x00000000000027d6, 0x00000000000027d7, 0x00000000000027d8, 0x00000000000027d9], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]  …  [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], 0.0, 1)

In [60]:
gmsh.finalize()